# Exchange example with SQL backend

In [162]:
import sqlite3
import datetime
import pandas as pd
from collections import OrderedDict

In [163]:
conn = sqlite3.connect('exchange.db')

In [164]:
c = conn.cursor()

In [189]:
def create_filled(bid, ask):
    
    users = list(OrderedDict(sorted(pd.read_sql_query('''SELECT * FROM users''', conn).to_dict(orient='index').items())).values())
    
    price = 0
    if bid['time'] < ask['time']:
        price = min(bid['price'],ask['price'])
    else:
        price = max(ask['price'],bid['price'])

    temp = {}
    temp['bid_id'] = bid['user_id']
    temp['ask_id'] = ask['user_id']
    temp['volume'] = min(bid['volume'],ask['volume'])
    temp['price'] = price
    temp['time'] = datetime.datetime.now()
    temp['bid_time'] = bid['time']
    temp['ask_time'] = ask['time']
    temp['security_id'] = bid['security_id']

    for user in users:
        if user['user_id'] == bid['user_id']:
            user['cash'] -= price * min(bid['volume'],ask['volume'])
        elif user['user_id'] == ask['user_id']:
            user['cash'] += price * min(bid['volume'],ask['volume'])
        else:
            pass
    
    users_df = pd.DataFrame(users)
    users_df.to_sql(name='users', con=conn, if_exists='replace', index = False)
    
    execute = 'UPDATE ref_prices SET ref_price = {} WHERE security_id = {}'.format(price,temp['security_id'])
    c.execute(execute)

    return temp

In [190]:
def validate_pin(user, pin):
    
    users = pd.read_sql_query('''SELECT * FROM users''', conn)

    if user not in list(users.username):
        return False
    else:
        if users[users.username == user].pin.iloc[0] == pin:
            return True
        else:
            return False

In [191]:
c.execute('DROP TABLE bids')
c.execute('DROP TABLE asks')
c.execute('DROP TABLE fills')
c.execute('DROP TABLE users')
c.execute('DROP TABLE markets')
c.execute('DROP TABLE positions')
c.execute('DROP TABLE ref_prices')
c.execute('DROP TABLE settlement')

In [192]:
c.execute('''CREATE TABLE bids
             (security_id, user_id, volume, price, time)''')

c.execute('''CREATE TABLE asks
             (security_id, user_id, volume, price, time)''')

c.execute('''CREATE TABLE fills
             (bid_id, ask_id, volume, price, time, bid_time, ask_time, security_id)''')

c.execute('''CREATE TABLE users
             (user_id, username, pin, cash)''')

c.execute('''CREATE TABLE markets
             (security_id, market_name, market_descriptor, create_time, end_time)''')

c.execute('''CREATE TABLE positions
             (security_id, user_id, position)''')

c.execute('''CREATE TABLE ref_prices
             (security_id, ref_price)''')

c.execute('''CREATE TABLE settlement
             (security_id, settle, in_settle)''')

# Create new users

In [193]:
def create_user(user, pin):
    users = pd.read_sql_query('''SELECT * FROM users''', conn)
    
    if user in list(pd.read_sql_query('''SELECT * FROM users''', conn).username):
        return 'Username taken'
    else:
        if len(users) > 0:
            user_id = int(users.user_id.max()) + 1
        else:
            user_id = 0

        exec_string = 'INSERT INTO users (user_id, username, pin, cash) values (?, ?, ?, ?)'
        c.execute(exec_string,
            (user_id, user, pin, 0))

In [194]:
create_user('Michael',1)
create_user('John',1)

In [195]:
pd.read_sql_query('''SELECT * FROM users''', conn)

,user_id,username,pin,cash
0,0,Michael,1,0
1,1,John,1,0


# Creating new markets

In [196]:
def create_market(market_name,market_descriptor, end_time):
    markets = pd.read_sql_query('''SELECT * FROM markets''', conn)
    if len(markets) > 0:
        security_id = int(markets.security_id.max()) + 1
    else:
        security_id = 0
    
    if len(markets.loc[markets.market_name == market_name]) > 0:
        return 'Market already exists'
    else: 
        exec_string = 'INSERT INTO markets (security_id, market_name, market_descriptor, create_time, end_time) values (?, ?, ?, ?, ?)'
        create_time = datetime.datetime.now()
        c.execute(exec_string,
            (security_id, market_name, market_descriptor, create_time, end_time))

        exec_string = 'INSERT INTO settlement (security_id,settle,in_settle) values ({},NULL,NULL)'.format(security_id)
        c.execute(exec_string)
        
        exec_string = 'INSERT INTO ref_prices (security_id,ref_price) values ({},NULL)'.format(security_id)
        c.execute(exec_string)
        

In [197]:
end_time = datetime.datetime(2020, 11, 17, 23, 59) #Dec 31st, 11:59 pm
create_market('Fake Market','Binary payout - contract size $1', end_time)

In [198]:
pd.read_sql_query('''SELECT * FROM markets''', conn).sort_values('create_time')

,security_id,market_name,market_descriptor,create_time,end_time
0,0,Fake Market,Binary payout - contract size $1,2020-04-22 21:24:22.592543,2020-11-17 23:59:00


# Create bid

In [199]:
def create_bid(security, user, pin, volume, price):
    
    close_markets()
    
    users = pd.read_sql_query('''SELECT * FROM users''', conn)
    
    if user not in list(users.username):
        return 'User does not exist'
    else:
        user_id = int(users[users.username == user]['user_id'].iloc[0])
    
    if security not in pd.read_sql_query('''SELECT * FROM markets''', conn).security_id:
        return 'Security does not exist'
    
    if user_id in list(pd.read_sql_query('''SELECT * FROM asks''', conn)['user_id']):
        asks = pd.read_sql_query('''SELECT * FROM asks''', conn)
        if price >= asks.loc[asks.user_id == user_id].price.max():
            return 'Invalid Order - crossing own ask'
    
    if security in pd.read_sql_query('''SELECT * FROM settlement WHERE in_settle = 1''', conn).security_id:
        return 'Market is closed'
    
    if users[users.username == user].pin.iloc[0] == pin:
    
        exec_string = 'INSERT INTO bids (security_id, user_id, volume, price, time) values (?, ?, ?, ?, ?)'
        time = datetime.datetime.now()
        c.execute(exec_string,
            (security, user_id, volume, price, time))

        order_flow()
        update_positions()
    
    else:
        return 'Incorrect Pin'

In [200]:
create_bid(0,'Michael',1,10,0.5)

In [201]:
pd.read_sql_query('''SELECT * FROM bids''', conn)

,price,security_id,time,user_id,volume
0,0.5,0,2020-04-22 21:24:24.389430,0,10


# Create Ask

In [202]:
def create_ask(security, user, pin, volume, price):
    
    close_markets()
    
    if user not in list(users.username):
        return 'User does not exist'
    else:
        user_id = int(users[users.username == user]['user_id'].iloc[0])
    
    if security not in pd.read_sql_query('''SELECT * FROM markets''', conn).security_id:
        return 'Security does not exist'
    
    if user_id in pd.read_sql_query('''SELECT * FROM bids''', conn).user_id:
        bids = pd.read_sql_query('''SELECT * FROM bids''', conn)
        if price <= bids.loc[bids.user_id == user_id].price.max():
            return 'Invalid Order - crossing own bid'
        
    if security in pd.read_sql_query('''SELECT * FROM settlement WHERE in_settle = 1''', conn).security_id:
        return 'Market is closed'
    
    if users[users.username == user].pin.iloc[0] == pin:
    
        exec_string = 'INSERT INTO asks (security_id, user_id, volume, price, time) values (?, ?, ?, ?, ?)'
        time = datetime.datetime.now()
        c.execute(exec_string,
            (security, user_id, volume, price, time))

        order_flow()
        update_positions()
    
    else:
        return 'Incorrect Pin'

In [203]:
create_ask(0,'John',1,10,0.5)

In [204]:
pd.read_sql_query('''SELECT * FROM asks''', conn)

,security_id,user_id,volume,price,time


# Delete exposure

In [205]:
def delete_exposure(security, user, pin):
    
    close_markets()
    
    if user not in list(users.username):
        return 'User does not exist'
    else:
        user_id = int(users[users.username == user]['user_id'].iloc[0])
    
    if security not in pd.read_sql_query('''SELECT * FROM markets''', conn).security_id:
        return 'Security does not exist'

    if security in pd.read_sql_query('''SELECT * FROM settlement WHERE in_settle = 1''', conn).security_id:
        return 'Market is closed'
    
    if users[users.username == user].pin.iloc[0] == pin:

        execute = 'DELETE FROM bids WHERE security_id = {} AND user_id = {}'.format(security,user_id)
        c.execute(execute)

        execute = 'DELETE FROM asks WHERE security_id = {} AND user_id = {}'.format(security,user_id)
        c.execute(execute)
    
        order_flow()
        update_positions()
    
    else:
        return 'Incorrect Pin'

In [206]:
delete_exposure(0,'Michael',1)

# Leaderboard

In [244]:
def ret_leaderboard():
    positions_df = pd.read_sql_query('''SELECT * FROM positions''', conn)
    ref_prices_df = pd.read_sql_query('''SELECT * FROM ref_prices''', conn)
    merged = positions_df.merge(ref_prices_df, on = ['security_id'])
    merged['position_worth'] = merged.position * merged.ref_price

    temp_list = []

    for index, row in pd.read_sql_query('''SELECT * FROM users''', conn).iterrows():

        temp = {}
        temp['username'] = row.username
        temp['pnl'] = row.cash + merged[merged.user_id == row.user_id].position_worth.sum()
        temp_list.append(temp)

    return pd.DataFrame(temp_list)[['username','pnl']].sort_values('pnl',ascending = False)

In [245]:
ret_leaderboard()

,username,pnl
0,Michael,0.0
1,John,0.0


# Run orderflow

In [18]:
def order_flow():
    
    bids = list(OrderedDict(sorted(pd.read_sql_query('''SELECT * FROM bids''', conn).sort_values('time').to_dict(orient='index').items())).values())
    asks = list(OrderedDict(sorted(pd.read_sql_query('''SELECT * FROM asks''', conn).sort_values('time').to_dict(orient='index').items())).values())

    for bid in bids:
        bid_id = bid['user_id']
        bid_volume = bid['volume']
        bid_price = bid['price']
        bid_time = bid['time']
        for ask in asks:
            ask_id = ask['user_id']
            ask_volume = ask['volume']
            ask_price = ask['price']
            ask_time = ask['time']

            if bid_id != ask_id and bid['security_id'] == ask['security_id']:
                if ask_price <= bid_price:
                    exec_string = 'INSERT INTO fills (bid_id, ask_id, volume, price, time, bid_time, ask_time, security_id) values (?, ?, ?, ?, ?, ?, ?, ?)'
                    temp = create_filled(bid, ask)
                    c.execute(exec_string,
                        (temp['bid_id'], temp['ask_id'], temp['volume'], temp['price'], temp['time'], temp['bid_time'], temp['ask_time'], temp['security_id']))
                    sub = min(bid['volume'],ask['volume'])
                    bid['volume'] -= sub
                    ask['volume'] -= sub

                    bids = list(filter(lambda i: i['volume'] != 0, bids)) 
                    asks = list(filter(lambda i: i['volume'] != 0, asks)) 

    bids = list(filter(lambda i: i['volume'] != 0, bids)) 
    asks = list(filter(lambda i: i['volume'] != 0, asks))
    
    if bids == []:
        c.execute('DELETE FROM bids')
    else:
        bids_df = pd.DataFrame(bids)
        bids_df.to_sql(name='bids', con=conn, if_exists='replace', index = False)
    
    if asks == []:
        c.execute('DELETE FROM asks')
    else:
        asks_df = pd.DataFrame(asks)
        asks_df.to_sql(name='asks', con=conn, if_exists='replace', index = False)        

# Get positions

In [115]:
def update_positions():
    fills = list(OrderedDict(sorted(pd.read_sql_query('''SELECT * FROM fills''', conn).to_dict(orient='index').items())).values())
    positions = []
    for fill in fills:
        temp_ask = {}
        temp_ask['security_id'] = fill['security_id']
        temp_ask['user_id'] = fill['ask_id']
        temp_ask['position'] = -fill['volume']

        temp_bid = {}
        temp_bid['security_id'] = fill['security_id']
        temp_bid['user_id'] = fill['bid_id']
        temp_bid['position'] = fill['volume']

        positions.append(temp_bid)
        positions.append(temp_ask)
    
    if positions == []:
        return
    else:
        positions_df = pd.DataFrame(positions)
        add_list = []
        for name, group in positions_df.groupby(['security_id','user_id']):
            temp = {}
            security_id = name[0]
            user_id = name[1]
            position = group.position.sum()
            temp['security_id'] = security_id
            temp['user_id'] = user_id
            temp['position'] = position
            add_list.append(temp)

        pd.DataFrame(add_list).to_sql(name='positions', con=conn, if_exists='replace', index = False)   

In [156]:
pd.read_sql_query('''SELECT * FROM positions''', conn)

,position,security_id,user_id
0,10,0,0
1,-10,0,1


# List of markets print out

In [230]:
def list_of_markets():
    
    list_of_marks = []
    
    fills_df = pd.read_sql_query('''SELECT * FROM fills''', conn)
    bids_df = pd.read_sql_query('''SELECT * FROM bids''', conn)
    asks_df = pd.read_sql_query('''SELECT * FROM asks''', conn)

    for index, row in pd.read_sql_query('''SELECT * FROM markets''', conn).iterrows():

        sendstr = ''

        if row.security_id in fills_df.security_id:
            sendstr += row.market_name + ' - Last traded price: ' + str(fills_df.sort_values('time').iloc[0].price)
        else:
            sendstr += row.market_name + ' - not traded yet'

        if row.security_id in bids_df.security_id:
            sendstr += ', Best bid: ' + str(bids_df.loc[bids_df.security_id == row.security_id].price.max())
        else:
            sendstr += ', No current bids'

        if row.security_id in asks_df.security_id:
            sendstr += ', Best ask: ' + str(asks_df.loc[asks_df.security_id == row.security_id].price.min())
        else:
            sendstr += ', No current asks'
            
        list_of_marks.append(sendstr)

    return list_of_marks

In [231]:
list_of_markets()

['Fake Market - Last traded price: 0.5, No current bids, No current asks']

# User summary

In [228]:
def get_user_info(user, pin):
    
    fills_df = pd.read_sql_query('''SELECT * FROM fills''', conn)

    if user not in list(users.username):
        return 'User does not exist'
    else:
        user_id = int(users[users.username == user]['user_id'].iloc[0])
        
    if users[users.username == user].pin.iloc[0] == pin:
    
        sendstr = 'Overall positions\n'
        user = pd.read_sql_query('''SELECT * FROM users WHERE user_id = {}'''.format(user_id), conn)
        sendstr += user.iloc[0].username + '\n'
        sendstr += 'Cash: ' + str(user.iloc[0].cash) + '\n'

        positions = pd.read_sql_query('''SELECT * 
                                         FROM positions 
                                         JOIN markets 
                                         ON positions.security_id = markets.security_id
                                         WHERE user_id = {}'''.format(user_id), conn)

        positions = positions.loc[:,~positions.columns.duplicated()]    
        
        for index, row in positions.iterrows():
            sendstr += row.market_name + ': ' + str(row.position) + ' contracts. Last traded price: ' + str(fills_df[fills_df.security_id == row.security_id].sort_values('time').iloc[0].price) + '\n\n'

        sendstr += 'Open orders\n'
        bids = pd.read_sql_query('''SELECT * 
                                     FROM bids 
                                     JOIN markets 
                                     ON bids.security_id = markets.security_id
                                     WHERE user_id = {}'''.format(user_id), conn)

        asks = pd.read_sql_query('''SELECT * 
                                 FROM asks 
                                 JOIN markets 
                                 ON asks.security_id = markets.security_id
                                 WHERE user_id = {}'''.format(user_id), conn)

        if len(bids) == 0:
            sendstr += 'No outstanding bids\n'
        else:
            sendstr += 'Outstanding bids\n'
            for index, row in bids.iterrows():
                sendstr += row.market_name + ' - ' + str(row.volume) + ' contracts for ' + str(row.price) + ' placed at: ' + str(row.time) + '\n'

        sendstr += '\n'

        if len(asks) == 0:
            sendstr += 'No outstanding asks\n'
        else:
            sendstr += 'Outstanding asks\n'
            for index, row in asks.iterrows():
                sendstr += row.market_name + ': ' + str(row.volume) + ' contracts for: ' + str(row.price) +'\n'

        return sendstr
    
    else:
        return 'Incorrect Pin'

In [229]:
print(get_user_info('Michael',1))

Overall positions
Michael
Cash: -5.0
Fake Market: 10 contracts. Last traded price: 0.5

Open orders
No outstanding bids

No outstanding asks



In [ ]:
create_bid(0,0,10,0.4)

In [ ]:
print(get_user_info(0))

# Settlement

In [ ]:
display(pd.read_sql_query('''SELECT * FROM markets''', conn))
display(pd.read_sql_query('''SELECT * FROM positions''', conn))
display(pd.read_sql_query('''SELECT * FROM settlement''', conn))
display(pd.read_sql_query('''SELECT * FROM bids''', conn))
display(pd.read_sql_query('''SELECT * FROM asks''', conn))
display(pd.read_sql_query('''SELECT * FROM users''', conn))
display(pd.read_sql_query('''SELECT * FROM fills''', conn))

In [16]:
def set_settle(security_id,settle):
    
    close_markets()
    
    execute = 'SELECT * FROM settlement WHERE security_id = {}'.format(security_id)
    temp_df = pd.read_sql_query(execute, conn)
    if len(temp_df.loc[temp_df.in_settle == 1]) == 0:
        return 'Market not closed'
    else:
        execute = 'UPDATE settlement SET settle = {} WHERE security_id = {}'.format(settle,security_id)
        c.execute(execute)

        pos = pd.read_sql_query('''SELECT * FROM positions JOIN settlement ON positions.security_id = settlement.security_id''', conn)
        for index, row in pos.iterrows():
            if row.in_settle == 1:
                execute = 'UPDATE users SET cash = cash + {} * {} WHERE user_id = {}'.format(row.position,row.settle,row.user_id)
                c.execute(execute)

        execute = 'DELETE FROM positions WHERE security_id = {}'.format(security_id)
        c.execute(execute)

In [15]:
def close_markets():
    for index, row in pd.read_sql_query('''SELECT * FROM markets''', conn).iterrows():
        if datetime.datetime.now() >= datetime.datetime.strptime(row.end_time,'%Y-%m-%d %H:%M:%S'):
            execute = 'UPDATE settlement SET in_settle = 1 WHERE security_id = {}'.format(row.security_id)
            c.execute(execute)
            execute = 'DELETE FROM bids WHERE security_id = {}'.format(row.security_id)
            c.execute(execute)
            execute = 'DELETE FROM asks WHERE security_id = {}'.format(row.security_id)
            c.execute(execute) 

In [ ]:
set_settle(0,1)

In [ ]:
display(pd.read_sql_query('''SELECT * FROM markets''', conn))
display(pd.read_sql_query('''SELECT * FROM positions''', conn))
display(pd.read_sql_query('''SELECT * FROM settlement''', conn))
display(pd.read_sql_query('''SELECT * FROM bids''', conn))
display(pd.read_sql_query('''SELECT * FROM asks''', conn))
display(pd.read_sql_query('''SELECT * FROM users''', conn))
display(pd.read_sql_query('''SELECT * FROM fills''', conn))